# Benchmarking

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_DEPOT_PATH = /home/manabu/.julia-1.7.3
  JULIA_NUM_THREADS = 12


---

In [2]:
using PyCall

In [6]:
ENV["VE_NODE_NUMBER"] = "0"

"0"

In [7]:
ENV["OMP_NUM_THREADS"] = "12"

"12"

In [8]:
ENV["MKL_ENABLE_INSTRUCTIONS"] = "AVX512"

"AVX512"

In [9]:
np = pyimport("numpy");
vp = pyimport("nlcpy");

---

In [10]:
const N = 2^27

134217728

In [11]:
# native
@time x0 = rand(Float64, (N,));
@time y0 = rand(Float64, (N,));

  0.171696 seconds (2 allocations: 1.000 GiB, 36.17% gc time)
  0.113920 seconds (2 allocations: 1.000 GiB, 3.83% gc time)


### axpy

In [14]:
pointer(x0), pointer(y0)

(Ptr{Float64} @0x00007f59eadf8040, Ptr{Float64} @0x00007f5a6adfa040)

In [15]:
# native
@time y0 += 3.14*x0;

  0.546139 seconds (435.19 k allocations: 2.023 GiB, 2.50% gc time, 18.88% compilation time)


In [16]:
pointer(x0), pointer(y0)

(Ptr{Float64} @0x00007f59eadf8040, Ptr{Float64} @0x00007f5aaadfb040)

厳密にaxpyではない･･･
* alloc時間が加算されている

### BLAS

In [21]:
using LinearAlgebra

In [24]:
pointer(x0), pointer(y0)

(Ptr{Float64} @0x00007f5a2adf9040, Ptr{Float64} @0x00007f5a6adfa040)

In [27]:
# native
@time LinearAlgebra.BLAS.axpy!(3.14, x0, y0);

  0.031929 seconds


In [26]:
pointer(x0), pointer(y0)

(Ptr{Float64} @0x00007f5a2adf9040, Ptr{Float64} @0x00007f5a6adfa040)

2-load/1-store, 2-flop:
* 94.0 GB/s
* 62.7 GFlop/s

In [30]:
3*1.0/0.031929, 2*1.0/0.031929

(93.95847035610261, 62.63898023740174)

### native code (loop vectorization)

In [31]:
#using Base.Threads: nthreads, @threads

In [12]:
#import Pkg; Pkg.add("LoopVectorization")
using LoopVectorization

In [13]:
Threads.nthreads()

12

In [15]:
@time @avxt for i in eachindex(x0)
    @inbounds y0[i] = 3.14*x0[i]
end

  0.038088 seconds (33 allocations: 784 bytes)


In [16]:
3*1.0/1.141157, 2*1.0/0.038088

(2.628910833478654, 52.50997689561017)

### Dot

In [51]:
@time _ = x0 ⋅ y0

  0.023695 seconds (1 allocation: 16 bytes)


3.3553767931433145e7

In [52]:
@time _ = LinearAlgebra.BLAS.dot(x0,y0)

  0.023541 seconds (1 allocation: 16 bytes)


3.3553767931433145e7

2-load/0-store, 2-flop:
* 84.4 GB/s
* 84.4 GFlop/s

In [53]:
2*1.0/0.023695

84.40599282549061

## Comparison

In [66]:
# PyArray (numpy)
@time x1 = @pycall np.random.random_sample((N,))::PyArray;
@time y1 = @pycall np.random.random_sample((N,))::PyArray;

  0.885777 seconds (19 allocations: 800 bytes)
  0.881678 seconds (19 allocations: 800 bytes)


In [57]:
@time x1 = @pycall np.asarray(x0)::PyArray;
@time y1 = @pycall np.asarray(x0)::PyArray;

  0.000055 seconds (19 allocations: 848 bytes)
  0.000039 seconds (19 allocations: 848 bytes)


In [58]:
blas = pyimport("scipy.linalg.blas");

In [59]:
typeof.((x1,y1))

(PyArray{Float64, 1}, PyArray{Float64, 1})

In [60]:
x1.data, y1.data

(Ptr{Float64} @0x00007f55156a1040, Ptr{Float64} @0x00007f55156a1040)

In [70]:
# PyArray (numpy)
@time _ = @pycall blas.daxpy(x1,y1,a=3.14)::PyArray;

  0.031510 seconds (35 allocations: 1.547 KiB)


In [71]:
@time z1 = @pycall np.dot(x1,y1)::PyAny

  0.024032 seconds (7 allocations: 256 bytes)


3.145122782445038e8

In [73]:
# nlcpy
@time x2 = vp.random.random_sample((N,));
@time y2 = vp.random.random_sample((N,));

  0.001031 seconds (8 allocations: 416 bytes)
  0.005262 seconds (8 allocations: 416 bytes)


In [75]:
@time x2 = vp.asarray(x1);
@time y2 = vp.asarray(y1);

  0.123860 seconds (4 allocations: 256 bytes)
  0.122843 seconds (4 allocations: 256 bytes)


In [76]:
(x2.ve_adr, y2.ve_adr) .|> (p)-> string(p, base=16)

("6101ec000010", "610230000010")

In [78]:
# nlcpy
@time _ = vp.dot(x2,y2)

  0.000176 seconds (7 allocations: 336 bytes)


PyObject array(3.14512278e+08)

In [148]:
(x2.ve_adr, y2.ve_adr) .|> (p)-> string(p, base=16)

("6100dc000010", "6101ec000010")

In [150]:
sizeof(x0), x1.o.nbytes, x2.nbytes

(1073741824, 1073741824, 1073741824)

In [80]:
2*1.0/0.000176/1024

11.097301136363637

11.1TFlop/s (11.1TB/s) は早すぎ･･･

## operator overload

In [52]:
@time z2 = @pycall np.add(x2,y2)::PyArray;

  0.251064 seconds (23 allocations: 992 bytes)


In [53]:
typeof(z2)

PyArray{Float64, 2}

## sum

In [66]:
# native
@time z0 = x0+y0;

  0.254086 seconds (2 allocations: 1.000 GiB, 1.20% gc time)


In [67]:
using LinearAlgebra

In [72]:
@time z00 = Base.copy(y0);
@time LinearAlgebra.BLAS.axpy!(1.,x0,z00);

  0.247239 seconds (2 allocations: 1.000 GiB, 1.23% gc time)
  0.031116 seconds


In [80]:
@time isapprox(z0, z00)

  0.541381 seconds (2 allocations: 1.000 GiB, 3.79% gc time)


true

In [15]:
# PyArray (numpy)
@time z2 = @pycall np.add(x2,y2)::PyArray;

  0.248329 seconds (23 allocations: 992 bytes)


In [34]:
typeof(z2)

PyArray{Float64, 2}

In [17]:
# nlcpy
@time z3 = x3+y3;

  0.001220 seconds (1 allocation: 16 bytes)


In [27]:
typeof(z3)

PyObject

## triad

In [30]:
# native
@time z0 = x0+3.14*y0;

  0.440315 seconds (4 allocations: 2.000 GiB)


In [35]:
# numpy
@time z2 = @pycall np.add(x2, @pycall np.multiply(3.14,y2)::PyArray)::PyArray;

  0.436774 seconds (51 allocations: 2.047 KiB)


In [32]:
# nlcpy
@time z3 = x3+3.14*y3;

  0.001912 seconds (3 allocations: 48 bytes)


### Prepare C-coded `triad()` for NLCPy

In [30]:
ve_lib = vp.jit.CustomVELibrary(
    code="""
        int ve_triad(double *px, double *py, double *pz, int n) {
            #pragma omp parallel for
            for (int i = 0; i  < n; i++) pz[i] = px[i] + 3.14*py[i];
            return 0;
        }
"""
)

PyObject <CustomVELibrary(
* code:
        int ve_triad(double *px, double *py, double *pz, int n) {
            #pragma omp parallel for
            for (int i = 0; i  < n; i++) pz[i] = px[i] + 3.14*py[i];
            return 0;
        }

* path: None
* cflags:  -c -fpic -O2 -I /opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/nlcpy/include -fopenmp
* ldflags:  -fpic -shared -fopenmp
* log_stream: None
* compiler: /opt/nec/ve/bin/ncc
* use_nlc: False
* ftrace: False
* ID: 3445858_2022-07-01.22:53:56.905138
* src_path: /tmp/tmp6_gd4acr/3445858_2022-07-01.22:53:56.905138.c
* obj_path: /tmp/tmp6_gd4acr/3445858_2022-07-01.22:53:56.905138.o
* lib_path: /tmp/tmp6_gd4acr/3445858_2022-07-01.22:53:56.905138.so
* lib: <nlcpy.veo._veo.VeoLibrary object at 0x7fd9bba06f90>
)>

In [31]:
ve_types = pyimport("nlcpy.ve_types");

In [32]:
ve_triad = ve_lib.get_function(
    "ve_triad",
    args_type=(ve_types.uint64, ve_types.uint64, ve_types.uint64, ve_types.int32),
    ret_type=ve_types.int32
)

PyObject <CustomVEKernel(func=<VeoFunction object VE function b've_triad'('uint64_t', 'uint64_t', 'uint64_t', 'int32_t') at 0x7fd96505a580>)>

In [33]:
#@time z0 = zeros(Float64, 10^6);
#@time z1 = @pycall np.zeros((10^6))::PyArray;
@time z2 = vp.zeros(10^6);

  0.000238 seconds (6 allocations: 288 bytes)


In [34]:
vp.shape(z2)

(1000000,)

In [36]:
# nlcpy w/c-coded triad
@time ve_triad(x2.ve_adr, y2.ve_adr, z2.ve_adr, z2.size, sync=true)

  0.000181 seconds (38 allocations: 1.203 KiB)


0